Before running this script, make sure that the metafeatures and metadataset files exist in the TabZilla directory

In [1]:
from pathlib import Path
import pandas as pd
from analysis_utils import get_tuned_alg_perf
import pdb, os, sys

root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

metadata_folder = Path("../TabZilla")

metadataset_df = pd.read_csv(metadata_folder / "tutorials/metadataset_new.csv")

# keep only binary and classification datasets. we have some results for regression datasets, which are not used.
metadataset_df = metadataset_df.loc[metadataset_df["target_type"].isin(["binary", "classification"]), :]

# read metafeatures
metafeatures_df = pd.read_csv(Path("../TabZilla/metafeatures.csv"))

# get the number of instances for each dataset, we will use these later
num_instances = metafeatures_df.loc[:, ["dataset_name", "f__pymfe.general.nr_inst"]]
num_instances.columns = ["dataset_fold_id", "num_inst"]

# make sure that the cleaned_results folder exists
output_folder = Path("./cleaned_results")
output_folder.mkdir(exist_ok=True)    

In [2]:
table_idx = 1

In [3]:
# merge in num instances - for runtime calculation
metadataset_df = metadataset_df.merge(num_instances, on="dataset_fold_id", how="left")

# calculate runtime
time_col = "training_time"

time_per_inst_col = "train_per_1000_inst"

metadataset_df.loc[:, time_per_inst_col] = 1000. * metadataset_df[time_col] / metadataset_df["num_inst"]

In [4]:
def get_datasets(table_idx):
    if table_idx == 1:
        from analysis.table1 import DATASETS
    elif table_idx == 2:
        from analysis.table2 import DATASETS
    elif table_idx == 4:
        from analysis.table4 import DATASETS
        
    return DATASETS

In [5]:
# metadataset_df = metadataset_df[metadataset_df.dataset_name.isin(get_datasets(table_idx))]

### Print number of results per dataset and alg

In [6]:
# for each alg, for how many datasets are there results?
print(f"for each alg: number of datasets with results (out of {len(metadataset_df['dataset_name'].unique())})")
print(metadataset_df.groupby("alg_name")["dataset_name"].apply(lambda x: len(set(x))).sort_values())

print(f"for each dataset: number of algs with results (out of {len(metadataset_df['alg_name'].unique())})")
metadataset_df.groupby("dataset_name")["alg_name"].apply(lambda x: len(set(x))).sort_values()

for each alg: number of datasets with results (out of 176)
alg_name
TabPFNModel            63
NAM                    80
DeepFM                 90
Ours                  115
TabTransformer        124
SAINT                 138
NODE                  141
SVM                   143
DANet                 147
rtdl_FTTransformer    148
VIME                  163
STG                   164
LightGBM              165
CatBoost              165
KNN                   167
LinearModel           168
TabNet                168
RandomForest          173
XGBoost               174
rtdl_ResNet           174
MLP                   175
DecisionTree          175
rtdl_MLP              176
Name: dataset_name, dtype: int64
for each dataset: number of algs with results (out of 23)


dataset_name
openml__poker-hand__9890             6
openml__covertype__7593              7
openml__Devnagari-Script__167121     8
openml__albert__189356              10
openml__CIFAR_10__167124            10
                                    ..
openml__cylinder-bands__14954       23
openml__credit-approval__29         23
openml__socmob__3797                23
openml__colic__27                   23
openml__kc2__3913                   23
Name: alg_name, Length: 176, dtype: int64

# A. Dataset inclusion/exclusion

**In this notebook: selected-18-algs:**
* We use a list of 18 algs (excluding 3 that had lots of errors.)
* We take only the datasets where each of these algs produce a result. This is ~100 datasets.

In [7]:
print(f"for each dataset: number of algs with results")
alg_counts = metadataset_df.groupby("dataset_name")["alg_name"].agg(lambda x: len(set(x))).sort_values()
print(alg_counts.head())

for each dataset: number of algs with results
dataset_name
openml__poker-hand__9890             6
openml__covertype__7593              7
openml__Devnagari-Script__167121     8
openml__albert__189356              10
openml__CIFAR_10__167124            10
Name: alg_name, dtype: int64


In [8]:
# drop:
# - TabPFN (only runs for small datasets)
# - NAM (lots of errors, long runtime)
# - DeepFM (not implemented for multi-class)
# - TabTransformer (lots of bugs...)

# selected_algs = [
#     "SAINT",
#     "NODE",
#     "SVM",
#     "DANet",
#     "rtdl_FTTransformer",
#     "VIME",
#     "STG",
#     "CatBoost",
#     "LightGBM",
#     "KNN",
#     "LinearModel",
#     "TabNet",
#     "RandomForest",
#     "XGBoost",
#     "rtdl_ResNet",
#     "MLP",
#     "DecisionTree",
#     "rtdl_MLP",
#     "NAM",
#     "TabPFNModel", # not included in this version
#     "DeepFM",
#     "TabTransformer", 
# ]

# test_df = metadataset_df.loc[metadataset_df["alg_name"].isin(selected_algs), :]

# # keep only datasets where all selected algs produce a result
# alg_count = test_df.groupby("dataset_name")["alg_name"].apply(lambda x: len(set(x)))

# keep_datasets = alg_count[alg_count == len(selected_algs)].index

# print(f"keeping {len(keep_datasets)} datasets")

# keep_df = test_df.loc[test_df["dataset_name"].isin(keep_datasets), :]

keep_df = metadataset_df

### Slightly more data prep

Note: We will keep all algs, regardless of how many datasets they have results for.

In [9]:
analysis_df = keep_df

print("after removing datasets: number of datasets with results")
dataset_counts = analysis_df.groupby("alg_name")["dataset_name"].agg(lambda x: len(set(x))).sort_values()
print(dataset_counts)

after removing datasets: number of datasets with results


alg_name
TabPFNModel            63
NAM                    80
DeepFM                 90
Ours                  115
TabTransformer        124
SAINT                 138
NODE                  141
SVM                   143
DANet                 147
rtdl_FTTransformer    148
VIME                  163
STG                   164
LightGBM              165
CatBoost              165
KNN                   167
LinearModel           168
TabNet                168
RandomForest          173
XGBoost               174
rtdl_ResNet           174
MLP                   175
DecisionTree          175
rtdl_MLP              176
Name: dataset_name, dtype: int64


# B. Tune and rank algorithms for each dataset

**Note**: At this point, you should have a dataframe called `analysis_df`, which contains all results you want to include in the remainder of the analysis. 

The code below performs hyperparameter tuning & ranking of each alg, and writes four cleaned results files to the directory `./cleaned_results`. 

In [10]:
metric_list = [
    "Accuracy",
    "F1",
    "Log Loss",
    "AUC",
]

obj_type_list = [
    "maximize",
    "maximize",
    "minimize",
    "maximize",
]
result_df_dict = {}

### Bookkeeping

In [11]:
# replace alg name with display name
from analysis_utils import ALG_DISPLAY_NAMES, ALG_TYPES
analysis_df.loc[:, "alg_name"] = analysis_df["alg_name"].apply(lambda x: ALG_DISPLAY_NAMES[x])

# add alg type
analysis_df.loc[:, "alg_type"] = analysis_df["alg_name"].apply(lambda x: ALG_TYPES[x])

In [12]:
# add a copy of each "default" hparam row, to treat this as a separate alg
default_rows = analysis_df.loc[analysis_df["hparam_source"] == "default"].copy()
default_rows.loc[:, "alg_name"] = default_rows["alg_name"].apply(lambda x: x + " (default)")

# remove TabPFN and LinearModel, since these only have one hparam set
default_rows = default_rows.loc[~(default_rows["alg_name"].str.contains("TabPFNModel") | default_rows["alg_name"].str.contains("LinearModel")), :]

# append these to the metadataset
analysis_df_with_default = pd.concat([analysis_df, default_rows], ignore_index=True)

### produce cleaned results files

first, each algorithm is tuned for each dataset fold (10 folds per dataset), using all available hyperparameter samples. we then calculate the normalized and ranked performance for each algorithm over all datasets.

the following loop produces four files:
- `./cleaned_results/tuned_aggregated_results.csv`: performance of each tuned algorithm on each dataset, where performance is averaged over all 10 folds. 
- `./cleaned_results/tuned_fold_results.csv`: performance of each tuned algorithm on each dataset fold.
- `./cleaned_results/tuned_aggregated_results_with_default.csv`: same as `tuned_aggregated_results_with_default.csv`, but with the default hyperparameters of each dataset included as a separate algorithm
- `./cleaned_results/tuned_fold_results_with_default.csv`: same as `tuned_fold_results_with_default.csv`, but with the default hyperparameters of each dataset included as a separate algorithm

In [13]:
tuned_result_dfs = {}
for drop_default in [True, False]:
    for i, (metric, objective_type) in enumerate(zip(metric_list, obj_type_list)):

        test_metric_col = metric + "__test"

        if drop_default:
            df = analysis_df.copy()
        else:
            df = analysis_df_with_default.copy()

        tuned_alg_perf = get_tuned_alg_perf(df, metric=metric)

        # NOTE: this "tunes" each algorithm for each training fold separately. so each of the 10 folds might use different hparams.
        tuned_result_dfs[metric] = tuned_alg_perf

        ##############################
        ### STEP 1: TREAT EACH FOLD AS SEPARATE DATASET

        result_col = test_metric_col
        
        # for each dataset, find the min and max metrics over all tuned algs
        overall_bounds = tuned_alg_perf.groupby("dataset_fold_id").agg({result_col: ["min", "max"]}).reset_index()

        # rename the multiindex cols
        new_cols = []
        for c in overall_bounds.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        overall_bounds.columns = new_cols

        tuned_alg_perf = tuned_alg_perf.merge(overall_bounds, on="dataset_fold_id", how="left")

        # add normalized metric
        tuned_alg_perf.loc[:, "normalized_" + result_col] = (tuned_alg_perf[result_col] - tuned_alg_perf[result_col + "_min"]) / (tuned_alg_perf[result_col + "_max"] - tuned_alg_perf[result_col + "_min"])

        # rank all algs for each dataset
        ascending = False if objective_type == "maximize" else True
        
        tuned_alg_perf.loc[:, f"{metric}_rank"] = tuned_alg_perf.groupby(["dataset_fold_id"])[result_col].rank(method="min", ascending=ascending).values

        # keep these cols to merge
        merge_cols = [
            "alg_name", 
            "dataset_fold_id", 
            "normalized_" + result_col,
            f"{metric}_rank",
            result_col + "_min",
            result_col + "_max"
        ]

        if i == 0:
            fold_tuned_df = tuned_alg_perf.copy()
        else:
            fold_tuned_df = fold_tuned_df.merge(tuned_alg_perf[merge_cols], on=["alg_name", "dataset_fold_id"])

        ##############################
        ### STEP 2: AVERAGE OVER FOLDS

        if i == 0:
            agg_dict = {
                test_metric_col: ["median", "mean"],
                "training_time": ["median", "mean"],
                time_per_inst_col: ["median", "mean"],
                # "dataset_name": ["count"],
            }
        else:
            agg_dict = {
                test_metric_col: ["median", "mean"],
                time_per_inst_col: ["median", "mean"],
            }

        # aggregate over folds: take the mean & median performance over each fold
        agg_tuned_alg_perf = tuned_alg_perf.groupby(["alg_name", "dataset_name"]).agg(agg_dict).reset_index()
        
        # rename the multiindex cols
        new_cols = []
        for c in agg_tuned_alg_perf.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        agg_tuned_alg_perf.columns = new_cols


        # define the target metric column, we will use this value for all plots
        result_col = test_metric_col + "_mean"

        # for each dataset, find the min and max metrics over all tuned algs
        overall_bounds = agg_tuned_alg_perf.groupby("dataset_name").agg({result_col: ["min", "max"]}).reset_index()
        
        # # normalize metric using these bounds, and get stddev of the mean normalized metric 
        # tuned_alg_perf = tuned_alg_perf.merge(overall_bounds, on="dataset_name", how="left")
        # tuned_alg_perf.loc[:, "norm_metric_tmp"] = (tuned_alg_perf[test_metric_col] -  tuned_alg_perf[result_col + "_min"]) / (tuned_alg_perf[result_col + "_max"] - tuned_alg_perf[result_col + "_min"])
        # std_metric = tuned_alg_pref.groupby() ...

        # rename the multiindex cols
        new_cols = []
        for c in overall_bounds.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        overall_bounds.columns = new_cols

        
        agg_tuned_alg_perf = agg_tuned_alg_perf.merge(overall_bounds, on="dataset_name", how="left")

        # add normalized metric
        agg_tuned_alg_perf.loc[:, "normalized_" + result_col] = (agg_tuned_alg_perf[result_col] - agg_tuned_alg_perf[result_col + "_min"]) / (agg_tuned_alg_perf[result_col + "_max"] - agg_tuned_alg_perf[result_col + "_min"])

        ###### - new - ######
        # estimate the standard deviation of the normalized metric by:
        # 1) normalize the metric for all folds using the normalization here
        # 2) get std of metric over all folds, like we did to get mean and median above
        tmp_df = tuned_alg_perf.loc[:, ["dataset_name", "alg_name", test_metric_col]].copy()
        tmp_df = tmp_df.merge(overall_bounds, on="dataset_name", how="left")    
        tmp_df.loc[:, "normalized_" + test_metric_col] = (tmp_df[test_metric_col] - tmp_df[result_col + "_min"]) / (tmp_df[result_col + "_max"] - tmp_df[result_col + "_min"])
        tmp_agg_df = tmp_df.groupby(["dataset_name", "alg_name"]).agg({"normalized_" + test_metric_col: "std"}).reset_index()
        tmp_agg_df.columns = ["dataset_name", "alg_name", "normalized_" + test_metric_col + "_std"]
        agg_tuned_alg_perf = agg_tuned_alg_perf.merge(tmp_agg_df, on=["dataset_name", "alg_name"], how="left")

        # rank all algs for each dataset
        ascending = False if objective_type == "maximize" else True
        
        # rank according to mean performance over all folds
        agg_method = "mean"

        # rank everything
        agg_tuned_alg_perf.loc[:, f"{metric}_rank_{agg_method}"]  = \
            agg_tuned_alg_perf.groupby(["dataset_name"])[test_metric_col + "_" + agg_method].rank(method="min", ascending=ascending).values

        agg_tuned_alg_perf.rename(columns={
            time_per_inst_col + "_median": time_per_inst_col + "_median_" + metric,
            time_per_inst_col + "_mean": time_per_inst_col + "_mean_" + metric,
        }, inplace=True)

        # keep these cols to merge
        merge_cols = [
            "alg_name", 
            "dataset_name",
            "normalized_" + result_col,
            "normalized_" + test_metric_col + "_std",
            time_per_inst_col + "_median_" + metric,
            time_per_inst_col + "_mean_" + metric,
            f"{metric}_rank_mean",
            result_col,
            result_col + "_min",
            result_col + "_max"
        ]

        if i == 0:
            tuned_agg_df = agg_tuned_alg_perf.copy()
        else:
            tuned_agg_df = tuned_agg_df.merge(agg_tuned_alg_perf[merge_cols], on=["alg_name", "dataset_name"])

    # save results

    # merge in alg type, for bookkeeping
    alg_type_df = analysis_df[["alg_name", "alg_type"]].drop_duplicates()
    tuned_agg_df = tuned_agg_df.merge(alg_type_df, on="alg_name", how="left")
    fold_tuned_df = fold_tuned_df.merge(alg_type_df, on="alg_name", how="left")

    if drop_default:
        agg_df_no_default = tuned_agg_df.copy()
        agg_df_no_default.to_csv("./cleaned_results/tuned_aggregated_results.csv")

        tuned_fold_df_no_default = fold_tuned_df.copy()
        tuned_fold_df_no_default.to_csv("./cleaned_results/tuned_fold_results.csv")
       
    else:
        agg_df_with_default = tuned_agg_df.copy()
        agg_df_with_default.to_csv("./cleaned_results/tuned_aggregated_results_with_default.csv")

        tuned_fold_df_with_default = fold_tuned_df.copy()
        tuned_fold_df_with_default.to_csv("./cleaned_results/tuned_fold_results_with_default.csv")


In [14]:
# take a peek
tuned_agg_df.head()

,alg_name,dataset_name,Accuracy__test_median,Accuracy__test_mean,training_time_median,training_time_mean,train_per_1000_inst_median_Accuracy,train_per_1000_inst_mean_Accuracy,Accuracy__test_mean_min,Accuracy__test_mean_max,...,Log Loss__test_mean_max,normalized_AUC__test_mean,normalized_AUC__test_std,train_per_1000_inst_median_AUC,train_per_1000_inst_mean_AUC,AUC_rank_mean,AUC__test_mean,AUC__test_mean_min,AUC__test_mean_max,alg_type
0,CatBoost,openml__APSFailure__168868,0.994145,0.994013,6.412328,7.276401,0.105466,0.119678,0.970303,0.994500,...,0.624879,1.000000,0.004881,0.186996,0.143336,1.0,0.991724,0.500000,0.991724,gbdt
1,CatBoost,openml__Amazon_employee_access__34539,0.946903,0.947359,1.708439,1.729567,0.065169,0.065976,0.928927,0.952150,...,1.020650,1.000000,0.035643,0.065087,0.075518,1.0,0.862513,0.492269,0.862513,gbdt
2,CatBoost,openml__Australian__146818,0.869565,0.872464,1.347650,1.393643,2.441396,2.524716,0.601449,0.872464,...,1.682198,0.992585,0.100835,2.114068,1.998640,2.0,0.942324,0.759653,0.943688,gbdt
3,CatBoost,openml__Bioresponse__9910,0.798940,0.795521,5.815126,6.748842,1.937729,2.248931,0.490545,0.796848,...,0.967133,0.971437,0.050844,2.237972,2.143907,5.0,0.865652,0.495182,0.876545,gbdt
4,CatBoost,openml__Census-Income__168340,0.958869,0.958658,830.825629,857.921920,3.470058,3.583213,0.943238,0.958658,...,0.474823,0.992560,0.018931,4.932732,4.874733,3.0,0.955365,0.852212,0.956139,gbdt


In [15]:
from analysis.table4 import DATASETS

tuned_agg_df[tuned_agg_df["dataset_name"].isin(DATASETS)][tuned_agg_df['alg_name'].isin(['CatBoost', 'TabFlex', 'TabPFN'])]

/tmp/ipykernel_4094171/525844158.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tuned_agg_df[tuned_agg_df["dataset_name"].isin(DATASETS)][tuned_agg_df['alg_name'].isin(['CatBoost', 'TabFlex', 'TabPFN'])]


,alg_name,dataset_name,Accuracy__test_median,Accuracy__test_mean,training_time_median,training_time_mean,train_per_1000_inst_median_Accuracy,train_per_1000_inst_mean_Accuracy,Accuracy__test_mean_min,Accuracy__test_mean_max,...,Log Loss__test_mean_max,normalized_AUC__test_mean,normalized_AUC__test_std,train_per_1000_inst_median_AUC,train_per_1000_inst_mean_AUC,AUC_rank_mean,AUC__test_mean,AUC__test_mean_min,AUC__test_mean_max,alg_type
2,CatBoost,openml__Australian__146818,0.869565,0.872464,1.347650,1.393643,2.441396,2.524716,0.601449,0.872464,...,1.682198,0.992585,0.100835,2.114068,1.998640,2.0,0.942324,0.759653,0.943688,gbdt
3,CatBoost,openml__Bioresponse__9910,0.798940,0.795521,5.815126,6.748842,1.937729,2.248931,0.490545,0.796848,...,0.967133,0.971437,0.050844,2.237972,2.143907,5.0,0.865652,0.495182,0.876545,gbdt
7,CatBoost,openml__GesturePhaseSegmentationProcessed__14969,0.615502,0.616022,11.644987,7.844962,1.474330,0.993208,0.298795,0.699889,...,5.715575,0.899894,0.019957,0.879405,0.800541,4.0,0.862134,0.500000,0.902419,gbdt
13,CatBoost,openml__MiniBooNE__168335,0.942757,0.942813,8.844148,7.955771,0.084998,0.076460,0.747838,0.946226,...,0.623800,0.985543,0.016432,0.085092,0.085040,4.0,0.984494,0.907805,0.985619,gbdt
16,CatBoost,openml__SpeedDating__146607,0.867542,0.866318,20.614545,34.475734,3.075481,5.143934,0.799234,0.871090,...,1.027214,0.959466,0.059792,6.341648,10.479124,5.0,0.869733,0.679571,0.877767,gbdt
18,CatBoost,openml__ada_agnostic__3896,0.855422,0.854230,0.406390,0.986107,0.111373,0.270177,0.751864,0.857521,...,1.003083,1.000000,0.067645,0.084726,0.183536,1.0,0.906147,0.667600,0.906147,gbdt
21,CatBoost,openml__airlines__189354,0.663289,0.663061,6.920802,6.949461,0.016039,0.016105,0.569102,0.671467,...,1.075940,0.933155,0.013691,0.016039,0.016105,4.0,0.714689,0.573717,0.724788,gbdt
22,CatBoost,openml__albert__189356,0.703379,0.704762,33.092527,35.153920,0.097276,0.103336,0.563360,0.704762,...,0.679891,1.000000,0.013941,0.096953,0.097484,1.0,0.776825,0.593685,0.776825,gbdt
29,CatBoost,openml__artificial-characters__14964,0.856164,0.854570,12.514503,12.465605,1.530976,1.524957,0.317188,0.954493,...,6.514097,0.953854,0.007902,1.530976,1.524957,5.0,0.988423,0.790613,0.997992,gbdt
30,CatBoost,openml__audiology__7,0.739130,0.752372,5.370374,159.025557,29.753904,878.758593,0.260474,0.800988,...,8.816495,0.814579,0.234645,18.259061,1098.535194,17.0,0.877642,0.531318,0.956476,gbdt


In [16]:
tuned_agg_df[tuned_agg_df["dataset_name"].isin(["openml__MiniBooNE__168335"])][tuned_agg_df['alg_name'].isin(['CatBoost', 'TabFlex', 'TabPFN'])]

/tmp/ipykernel_4094171/1667309541.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tuned_agg_df[tuned_agg_df["dataset_name"].isin(["openml__MiniBooNE__168335"])][tuned_agg_df['alg_name'].isin(['CatBoost', 'TabFlex', 'TabPFN'])]


,alg_name,dataset_name,Accuracy__test_median,Accuracy__test_mean,training_time_median,training_time_mean,train_per_1000_inst_median_Accuracy,train_per_1000_inst_mean_Accuracy,Accuracy__test_mean_min,Accuracy__test_mean_max,...,Log Loss__test_mean_max,normalized_AUC__test_mean,normalized_AUC__test_std,train_per_1000_inst_median_AUC,train_per_1000_inst_mean_AUC,AUC_rank_mean,AUC__test_mean,AUC__test_mean_min,AUC__test_mean_max,alg_type
13,CatBoost,openml__MiniBooNE__168335,0.942757,0.942813,8.844148,7.955771,0.084998,0.07646,0.747838,0.946226,...,0.6238,0.985543,0.016432,0.085092,0.08504,4.0,0.984494,0.907805,0.985619,gbdt


# merge in metafeatures and rewrite files

In [17]:
import pandas as pd
from pathlib import Path

# metafeatures_df = pd.read_csv(Path("metafeatures.csv"))

metafeatures_df.rename(columns={"dataset_name": "dataset_fold_id"}, inplace=True)

metafeatures_df.loc[:, "f__pymfe.general.total_num_instances"] = metafeatures_df["f__pymfe.general.nr_inst"] / 0.8
metafeatures_df.loc[:, "dataset_basename"] = metafeatures_df["dataset_fold_id"].str[:-len("__fold_1")]
agg_metafeatures = metafeatures_df.groupby("dataset_basename").median(numeric_only=True)

# remove histogram columns
keep_cols = [c for c in agg_metafeatures.columns if "hist" not in c]

# rename each column to be easier
# new_col_names = {
#     c: c.removeprefix("f__pymfe.") for c in agg_metafeatures.columns
# }
# agg_metafeatures.rename(columns=new_col_names, inplace=True)

round_attrs = [
    "f__pymfe.general.total_num_instances",
    "f__pymfe.general.nr_attr",
    "f__pymfe.general.nr_bin",
    "f__pymfe.general.nr_cat",
    "f__pymfe.general.nr_num",
    "f__pymfe.general.nr_class",
]
for attr in round_attrs:
    agg_metafeatures.loc[:, attr] = agg_metafeatures[attr].round(0)

agg_metafeatures.to_csv(output_folder / "agg_metafeatures.csv")


In [18]:
agg_metafeatures["f__pymfe.general.nr_inst"].describe()

count       183.000000
mean      31353.909836
std      105278.317705
min          26.000000
25%         490.500000
50%        2027.500000
75%       10747.000000
max      820007.000000
Name: f__pymfe.general.nr_inst, dtype: float64